In [1]:
import pandas as pd
import os
import numpy
import dicom2jpg
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize

C:\Users\munee\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
scores_df = pd.read_csv("Clinical Scores.csv")

In [3]:
scores_df

,Subject ID,Sex,Research Group,Visit,Age,MMSE Total Score,Global CDR,Description
0,003_S_1057,F,MCI,ADNI1/GO Month 6,62.2,25.0,0.5,2nd adni recon
1,003_S_1057,F,MCI,ADNI1/GO Month 12,62.6,28.0,0.5,PET adni3
2,003_S_1057,F,MCI,ADNI1/GO Month 24,63.6,23.0,0.5,PET ADNI
3,003_S_1057,F,MCI,ADNI1/GO Month 36,64.5,23.0,1.0,PET ADNI
4,003_S_1074,F,MCI,ADNI1/GO Month 6,86.0,29.0,0.5,ADNI
...,...,...,...,...,...,...,...,...
850,941_S_1194,M,CN,ADNI1/GO Month 36,88.0,30.0,0.5,ADNI Brain PET: Raw
851,941_S_1202,M,CN,ADNI1/GO Month 6,78.2,30.0,0.0,ADNI Brain PET: Raw
852,941_S_1202,M,CN,ADNI1/GO Month 12,78.7,30.0,0.0,ADNI Brain PET: Raw
853,941_S_1202,M,CN,ADNI1/GO Month 24,79.7,30.0,0.5,ADNI Brain PET: Raw


In [4]:
scores_df = scores_df.drop(columns = ["Sex", "Age", "Research Group", "Description"])

In [5]:
def load_image(path = "."):
    image = []
    for i in os.listdir(path):
        if i[-3:] == "dcm":
            image.append(dicom2jpg.dicom2img(path + "/" + i))
    image = np.array(image)
    image = resize(image, (81, 168, 168))
    return image

In [6]:
def preprocess(image):
    prepped_image = np.zeros((168, 186, 144))
    for slice in range(len(prepped_image)):
        test = np.rot90(image, k=1, axes=(2,0))[slice]
        test2 = test[20: 140]
        test3 = np.rot90(test2, k = 1, axes=(0,1))
        test4 = np.flipud(cv2.resize(np.rot90(test3, k = 1, axes=(1,0)), (144, 186)))
        prepped_image[slice] = test4
    prepped_image = prepped_image[8:-8]
    prepped_image = prepped_image[40:120]
    resized_image = resize(prepped_image, (152, 186, 144))
    return resized_image

In [7]:
subjects = []
i = 0
visits = ["ADNI1/GO Month 6", "ADNI1/GO Month 12", "ADNI1/GO Month 24", "ADNI1/GO Month 36"]
for path in os.listdir():
    if "." not in path and path != "137_S_1414":
        if path in scores_df["Subject ID"].unique():
            temp = scores_df.loc[scores_df["Subject ID"] == path]
            flag = 0
            for v in visits:
                if v not in temp["Visit"].unique():
                    flag = 1
                    break
            if not flag:
                subjects.append(path)
                i += 1
print(i)

79


In [8]:
images = np.zeros((79, 152, 186, 144), dtype='uint8')

In [9]:
scores = np.zeros((79, 8))

In [10]:
subjects

['003_S_1057',
 '003_S_1074',
 '003_S_1122',
 '007_S_0101',
 '007_S_0128',
 '007_S_0344',
 '007_S_0698',
 '012_S_0634',
 '012_S_0637',
 '012_S_1133',
 '013_S_0502',
 '016_S_0359',
 '016_S_1092',
 '021_S_0141',
 '021_S_0178',
 '021_S_0231',
 '021_S_0424',
 '021_S_0626',
 '022_S_0066',
 '022_S_0096',
 '022_S_0130',
 '022_S_0961',
 '027_S_0074',
 '027_S_0120',
 '027_S_0408',
 '033_S_0513',
 '033_S_0567',
 '033_S_0723',
 '033_S_0734',
 '033_S_0741',
 '033_S_0906',
 '037_S_0150',
 '037_S_0327',
 '037_S_0377',
 '037_S_0454',
 '037_S_0566',
 '037_S_1078',
 '052_S_1346',
 '057_S_0464',
 '057_S_0779',
 '057_S_0818',
 '057_S_0934',
 '057_S_1007',
 '057_S_1217',
 '057_S_1265',
 '062_S_1299',
 '099_S_0051',
 '099_S_0054',
 '099_S_0090',
 '099_S_0291',
 '099_S_0352',
 '099_S_0533',
 '099_S_0534',
 '099_S_0551',
 '100_S_0047',
 '100_S_0995',
 '114_S_0173',
 '114_S_0378',
 '114_S_0416',
 '114_S_1106',
 '114_S_1118',
 '130_S_0232',
 '130_S_0285',
 '130_S_0289',
 '136_S_0695',
 '137_S_0283',
 '137_S_03

In [11]:
scores_df.loc[(scores_df["Subject ID"] == "137_S_1414") & (scores_df["Visit"] == "ADNI1/GO Month 6")]

,Subject ID,Visit,MMSE Total Score,Global CDR
842,137_S_1414,ADNI1/GO Month 6,28.0,0.5
843,137_S_1414,ADNI1/GO Month 6,28.0,0.5


In [12]:
i = 0
for subject in subjects:
    if i%10 == 0:
        print(i)
    temp = scores_df.loc[scores_df["Subject ID"] == subject]
    images[i] = preprocess(load_image(subject))
    subject_scores = []
    for v in visits:
        subject_scores.append(temp.loc[temp["Visit"] == v]["MMSE Total Score"].to_numpy()[0])
        subject_scores.append(temp.loc[temp["Visit"] == v]["Global CDR"].to_numpy()[0])
    scores[i] = np.array(subject_scores)
    i += 1
print(i)

0


C:\Users\munee\anaconda3\lib\site-packages\dicom2jpg\utils.py:123: RuntimeWarning: invalid value encountered in divide
  pixel_array = ((pixel_array-pixel_array.min())/(pixel_array.max()-pixel_array.min())) * 255.0
C:\Users\munee\anaconda3\lib\site-packages\dicom2jpg\utils.py:130: RuntimeWarning: invalid value encountered in cast
  return pixel_array.astype('uint8')


10
20
30
40
50
60
70
79


In [13]:
# i = 0
# for path in os.listdir():
#     if "." not in path and path != "137_S_1414":
#         if path in scores_df["Subject ID"].unique():
#             if i%10 == 0:
#                 print(i)
#             images[i] = preprocess(load_image(path))
#             scores[i] = scores_df[scores_df["Subject ID"] == path].to_numpy()[0][1:]
#             i += 1

0


ValueError: could not broadcast input array from shape (3,) into shape (8,)

In [14]:
print(i)

0


In [15]:
print(numpy.isnan(images).any())
print(numpy.isnan(scores).any())

False
False


In [16]:
scores.shape

(79, 8)

In [17]:
scores

array([[25. ,  0.5, 28. ,  0.5, 23. ,  0.5, 23. ,  1. ],
       [29. ,  0.5, 29. ,  0.5, 29. ,  0.5, 27. ,  0. ],
       [29. ,  0.5, 29. ,  0.5, 30. ,  0.5, 30. ,  0.5],
       [29. ,  0.5, 27. ,  0.5, 28. ,  0.5, 25. ,  0.5],
       [27. ,  0.5, 29. ,  0.5, 25. ,  1. , 26. ,  1. ],
       [24. ,  0.5, 22. ,  0.5, 23. ,  1. , 20. ,  1. ],
       [29. ,  0.5, 29. ,  0.5, 29. ,  0.5, 30. ,  0.5],
       [26. ,  0.5, 29. ,  0.5, 26. ,  0.5, 30. ,  0.5],
       [30. ,  0. , 29. ,  0.5, 30. ,  0. , 30. ,  0. ],
       [29. ,  0. , 30. ,  0. , 30. ,  0. , 30. ,  0. ],
       [30. ,  0. , 30. ,  0. , 28. ,  0. , 28. ,  0. ],
       [30. ,  0. , 29. ,  0. , 30. ,  0. , 30. ,  0. ],
       [29. ,  0.5, 29. ,  0.5, 29. ,  0. , 24. ,  0. ],
       [27. ,  0.5, 27. ,  1. , 23. ,  1. , 24. ,  1. ],
       [30. ,  0.5, 29. ,  0.5, 30. ,  0.5, 30. ,  0.5],
       [28. ,  0.5, 29. ,  0.5, 25. ,  1. , 27. ,  1. ],
       [25. ,  0.5, 27. ,  0.5, 28. ,  0.5, 27. ,  0.5],
       [23. ,  0.5, 25. ,  0.5,

In [21]:
np.save("images2", images)
np.save("scores2", scores)